In [1]:
#librerías 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

In [2]:

#Leemos el archivo meteo-nogal (las meteorológicas)
df1 = pd.DataFrame( pd.read_csv("meteo-nogal-09.csv", engine="python" ) )
df1.head()

FileNotFoundError: [Errno 2] No such file or directory: 'meteo-nogal-09.csv'

In [ ]:
#Mostramos los tipos de variables de nuestros datos anteriores

df1.dtypes

In [ ]:
#Si nos fijamos en los tipos anteriores la mayoría dice unnamed.
#Quitaremos todas las columnas que contengan a los "unnamed".

#La primera columna no nos está dando datos relevantes, sólo nos dice las unidades de medida y algunos NAN.
#Por lo que quitaremos la primer columna sin que se elimine el índice,
#haremos que el índice vuelva a contar desde cero ya que esté eliminada la columna.
df1=df1.drop(df1.columns[df1.columns.str.contains('unnamed:',case = False)],axis = 1)
df1 = df1.drop(df1.index[0]).reset_index(drop=True)
df1.head()

In [ ]:
#Volvemos a ver los tipos de las variables
df1.dtypes

In [ ]:
#Creamos una variable llamada Fecha0 apartit de Date y time.
#Que tenga al dato de Date , luego un espacio luego el dato de time.



#Como ya tenemos FECHA0 , entonces no sirve de mucho tener a DATE y TIME por separado.
#Eliminamos DATE y eliminamos TIME.

df1["FECHA0"] = df1["DATE"] +" "+ df1["TIME"]
df1.drop( ["DATE","TIME"], axis=1, inplace=True )

In [ ]:
#Queremos python que reconozca a Fecha0 como una variable de fecha.
#Creemos una variable llamada FECHA de tipo fechas a la que le asignamos el valor FECHA0. 
df1['FECHA'] = pd.to_datetime(df1.apply(lambda x: x['FECHA0'], 1), dayfirst=True)
#Eliminemos al anterior FECHA0
df1 = df1.drop(['FECHA0'], 1)
df1.head()

In [ ]:
#Como ya tenemos a Fecha como una variable tipo fecha, ya podemos separarla por componentes. 
#Creamos una columna de meses
df1['MES'] = df1['FECHA'].dt.month
#Creamos una columna de días
df1['DIA'] = df1['FECHA'].dt.day
#Creamos una columna de horas
df1['HORA'] = df1['FECHA'].dt.hour
#Creamos una columna de minutos
df1['MINUTO'] = df1['FECHA'].dt.minute
df1.head()

In [ ]:
#Aquí removeremos los dos primeros renglones, tomando en cuenta que 
#los data frames se parecen a partir del 3er renglón
df1 = df1.drop(df1.index[0:2]).reset_index(drop=True)
df1.head()

In [ ]:
#Tomaremos los datos cada 30 minutos
df1 = df1[((df1['MINUTO'] == 30.0) | (df1['MINUTO'] == 0.0) ) & (df1['FECHA']<'2010-01-01 00:30:00')]
#Hacemos que se recorran los índices
df1= df1.reset_index(drop=True)
df1.tail()

In [ ]:
#Ahora leeremos el otro archivo soil-nogal (El del suelo), creando un dataframe con el
df2 = pd.DataFrame( pd.read_csv("soil-nogal-09.csv", engine="python" ) )
df2.head()

In [ ]:
#De igual manera, revisamos el tipo de variables
df2.dtypes

In [ ]:
#Filtramos las columnas de interés
df2 = df2.filter(['2 Year_RTM  L','3 Day_RTM  L','4 Hour_Minute_RTM  L','Tsuelo_10cm','Tsuelo_20cm','Tsuelo_30cm','Tsuelo_40cm'
                  ,'Tsuelo_55cm','Tsuelo_70cm','Tsuelo_85cm','Tsuelo_100cm'],axis=1)
df2.head()

In [ ]:
#Formaremos una variable de fecha, haciendo dos arreglos con horas y minutos.
#Convertiremos '4 Hour_Minute_RTM  L' a tipo string.
df2['4 Hour_Minute_RTM  L'] = df2['4 Hour_Minute_RTM  L'].astype(str)

In [ ]:
#Haremos los arreglos de horas y minutos

hora=[]
minuto=[]

for i in range (0, len(df2)):
    #Si contiene 4 caracteres:
    if (len(str(df2['4 Hour_Minute_RTM  L'][i]))==4):
        #Revisamos si es el caso 2400:
        if (str(df2['4 Hour_Minute_RTM  L'][i])[0:2]=='24'):
            hora.append('00')
            minuto.append('00')
        else:
            hora.append(str(df2['4 Hour_Minute_RTM  L'][i])[0:2])
            minuto.append(str(df2['4 Hour_Minute_RTM  L'][i])[2:4])
    #Si resulta ser de 3 caracteres:
    elif (len(str(df2['4 Hour_Minute_RTM  L'][i]))==3):
            hora.append(str(df2['4 Hour_Minute_RTM  L'][i])[0:1])
            minuto.append(str(df2['4 Hour_Minute_RTM  L'][i])[1:3])
    #Si resulta ser de 2 caracteres:
    elif (len(str(df2['4 Hour_Minute_RTM  L'][i]))==2):
            hora.append('00')
            minuto.append(str(df2['4 Hour_Minute_RTM  L'][i])[0:2])

#Crearemos el arreglos de días
dias =[df2['3 Day_RTM  L'][i] for i in range(0,len(df2))]

In [ ]:
#Hacemos un data frame con las horas, minutos y días del segundo data frame
d = {'dias': dias, 'hora': hora, 'minuto':minuto}
df_fechas = pd.DataFrame(data=d)
df_fechas.head()

In [ ]:
#Para arreglar la fecha cuando la hora marca las 12am, aumentamos un día cuando el reloj marque tal hora 
dia=[]
for i in range(0,len(df_fechas)):
    if (df_fechas['hora'][i]=='00' and df_fechas['minuto'][i]=='00'):
        dia.append(df_fechas['dias'][i] +1)
    else:
        dia.append(df_fechas['dias'][i])
df_fechas['dia']=dia

In [ ]:
#Mostramos el daata frame
df_fechas.head()

In [ ]:
#Hacemos una variable string a partir de los datos del data frame
fechas = []
for i in range (0,len(df2)):
    fechas.append('2009 '+str(df_fechas['dia'][i])+ ' ' + df_fechas['hora'][i]+' '+df_fechas['minuto'][i])


In [ ]:
#Vamos a convertir los datos del arreglo anterior a variables datetime, tomando en cuenta que se trata del calendario juliano
FECHA = []
for i in range(0,len(df2)):
    d=datetime.datetime.strptime(fechas[i],'%Y %j %H %M')
    F = d.isoformat(' ')
    FECHA.append(F)

In [ ]:
#Ahora, agregaremos la columna FECHA0 al dataframe
df2['FECHA0']=FECHA
df2.tail()

In [ ]:
#Después vamos a convertir FECHA0 a datetime
df2['FECHA'] = pd.to_datetime(df2.apply(lambda x: x['FECHA0'], 1), dayfirst=True)
#Eliminamos FECHA0
df2 = df2.drop(['FECHA0'], 1)
df2.dtypes

In [ ]:
#Como los archivos pudieran tener datos repetidos, quitamos los duplicados de ambos adata frames
df2 = df2.drop_duplicates(subset=['FECHA'])
df1 = df1.drop_duplicates(subset=['FECHA'])

In [ ]:
#Finalmente unimos los data frames por medio de la variable FECHA
df3 = pd.merge(df1, df2, on=['FECHA'])

In [ ]:
#Mostramos el nuevo data frame
df3.head()

In [ ]:
#Eliminamos los datos que no buscamos grafica
df3 = df3.filter(['FECHA','Tsuelo_10cm','Tsuelo_20cm','Tsuelo_30cm','Tsuelo_40cm'
                  ,'Tsuelo_55cm','Tsuelo_70cm','Tsuelo_85cm','Tsuelo_100cm','t_Avg','air_Avg','net_rad_Avg','airT_Avg'],axis=1)
df3.head()

In [ ]:
#Elegimos un día de Enero y graficamos la temperatura del aire y las 4 temperaturas del subsuelo.
#Graficaremos las temperaturas del primero de enero, escogiendo los datos menores al 2 de enero
graf1=df3[df3['FECHA']<'2009-01-02 00:00:00']
graf1_s = graf1.filter(['Tsuelo_10cm','Tsuelo_20cm','Tsuelo_40cm',
                            'Tsuelo_85cm'],axis=1)

#Para poder operar numéricamente, convertimos las variables object a float64
graf1 = graf1.apply(pd.to_numeric, errors='coerce')
graf1_s.head()

In [ ]:
#Hacemos dos gráficas, una para las temperaturas del suelo ese día y otra para el aire

#Temperaturas del subsuelo par el 01/01/2009
sns.set(rc={'figure.figsize':(12,8.27)})

fig=plt.figure(); graf1_s.plot(); plt.legend(loc='best')
plt.title("Temperaturas el 1 de Enero")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("graf_1enero",plt=2000)

In [ ]:
#Temperatura del aire para el 01/01/2009
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); graf1["airT_Avg"].plot() ; plt.legend(loc='best')
plt.title("Temperaturas de 1 de Enero")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("Tairundia",plt=2000)

In [ ]:
#Realizaremos una gráfica de temperaturas T_max, T_min y T_promedio diarias para el año completo de datos 2009. 

#Usaremos un nuevo dataframe
graf2=df3

#Hacemos variables para día y mes
graf2['DIA']=graf2['FECHA'].dt.day
graf2['MES']=graf2['FECHA'].dt.month


#Hacemos un data frame para c/u de  las  temperaturas del suelo

#Para Tsuelo_10cm 
df_10=graf2.filter(['DIA','MES','Tsuelo_10cm'],axis=1)
#Calculamos la temperatura mínima, máxima y promedio
df_10["T_10cm_max"] = np.round(df_10.groupby(["MES","DIA"])["Tsuelo_10cm"].transform("max"),decimals=1)
df_10["T_10cm_min"] = np.round(df_10.groupby(["MES","DIA"])["Tsuelo_10cm"].transform("min"),decimals=1)
df_10["T_10cm_mean"] = np.round(df_10.groupby(["MES","DIA"])["Tsuelo_10cm"].transform("mean"),decimals=1)
#Eliminamos los datos que no se grafican
df_10 = df_10.drop(['Tsuelo_10cm','DIA','MES'], 1)

#Para Tsuelo_20cm 
df_20=graf2.filter(['DIA','MES','Tsuelo_20cm'],axis=1)
#Calculamos la temperatura mínima, máxima y promedio
df_20["T_20cm_max"] = np.round(df_20.groupby(["MES","DIA"])["Tsuelo_20cm"].transform("max"),decimals=1)
df_20["T_20cm_min"] = np.round(df_20.groupby(["MES","DIA"])["Tsuelo_20cm"].transform("min"),decimals=1)
df_20["T_20cm_mean"] = np.round(df_20.groupby(["MES","DIA"])["Tsuelo_20cm"].transform("mean"),decimals=1)
#Eliminamos los datos que no se grafican
df_20 = df_20.drop(['Tsuelo_20cm','DIA','MES'], 1)

#Para Tsuelo_40cm 
df_40=graf2.filter(['DIA','MES','Tsuelo_40cm'],axis=1)
#Calculamos la temperatura mínima, máxima y promedio
df_40["T_40cm_max"] = np.round(df_40.groupby(["MES","DIA"])["Tsuelo_40cm"].transform("max"),decimals=1)
df_40["T_40cm_min"] = np.round(df_40.groupby(["MES","DIA"])["Tsuelo_40cm"].transform("min"),decimals=1)
df_40["T_40cm_mean"] = np.round(df_40.groupby(["MES","DIA"])["Tsuelo_40cm"].transform("mean"),decimals=1)
#Eliminamos los datos que no se grafican
df_40 = df_40.drop(['Tsuelo_40cm','DIA','MES'], 1)

#Para Tsuelo_85cm 
df_85=graf2.filter(['DIA','MES','Tsuelo_85cm'],axis=1)
#Calculamos la temperatura mínima, máxima y promedio
df_85["T_85cm_max"] = np.round(df_85.groupby(["MES","DIA"])["Tsuelo_85cm"].transform("max"),decimals=1)
df_85["T_85cm_min"] = np.round(df_85.groupby(["MES","DIA"])["Tsuelo_85cm"].transform("min"),decimals=1)
df_85["T_85cm_mean"] = np.round(df_85.groupby(["MES","DIA"])["Tsuelo_85cm"].transform("mean"),decimals=1)
#Eliminamos los datos que no se grafican
df_85 = df_85.drop(['Tsuelo_85cm','DIA','MES'], 1)



In [ ]:
#Gráfica para Tsuelo_10cm
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); df_10.plot() ; plt.legend(loc='best')
plt.title("Temperatura del suelo a 10cm")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T10",plt=2000)

In [ ]:
#Gráfica para Tsuelo_20cm
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); df_20.plot() ; plt.legend(loc='best')
plt.title("Temperatura del suelo a 20cm")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T20",plt=2000)

In [ ]:
#Gráfica para Tsuelo_40cm
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); df_40.plot() ; plt.legend(loc='best')
plt.title("Temperatura del suelo a 40cm")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T40",plt=2000)

In [ ]:
#Gráfica para Tsuelo_85cm
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); df_85.plot() ; plt.legend(loc='best')
plt.title("Temperatura del suelo a 85cm")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T85",plt=2000)

In [ ]:
#Realizamos la gráfica para la temperatura del aire

#Quitaremos todas las variables excepto las necesarias para graficar el aire
graf2=graf2.filter(['DIA','MES','airT_Avg'],axis=1)

graf2.head()

#Convertimos los datos a float 64
graf2[graf2.columns[1:3]]=graf2[graf2.columns[1:3]].apply(pd.to_numeric,errors='coerce')
graf2.dtypes

In [ ]:
#Agrupamos y usamos la función transform para calcular las temperaturas máxima, mínima y promedio del aire
graf2["airT_Avg_max"] = np.round(graf2.groupby(["MES","DIA"])["airT_Avg"].transform("max"),decimals=1)
graf2["airT_Avg_min"] = np.round(graf2.groupby(["MES","DIA"])["airT_Avg"].transform("min"),decimals=1)
graf2["airT_Avg_mean"] = np.round(graf2.groupby(["MES","DIA"])["airT_Avg"].transform("mean"),decimals=1)

#Quitamos las columnas que no nos interesan para realizar la gráfica
graf2 = graf2.drop(['airT_Avg','DIA','MES'], 1)
graf2.head()


In [ ]:
#Elaboramos la gráfica de la temperatura del aire promedio, máxima y mínima por día para todo 2009
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); graf2.plot() ; plt.legend(loc='best')
plt.title("Temperatura del aire para el año 2009")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T_aire_2009",plt=2000)

In [ ]:
#Calcular el promedio cada 30 minutos durante el día para el mes de Enero de 
#la temperatura del aire y las 8 temperaturas promedio de subsuelo, para posteriormente
#graficar la variación en 24 horas de las temperaturas de interés.

#Tomamos el mes de enero
graf3=df3[df3['FECHA']<'2009-02-01 00:00:00']

#Tomamos las temperaturas del suelo
graf3_s = graf3.filter(['Tsuelo_10cm','Tsuelo_20cm','Tsuelo_30cm','Tsuelo_40cm',
                              'Tsuelo_55cm','Tsuelo_70cm','Tsuelo_85cm','Tsuelo_100cm'],axis=1)
graf3_s.head()

In [ ]:
#Gráfica de temperaturas del suelo cada 30 min para enero del 2009
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); graf3_s.plot() ; plt.legend(loc='best')
plt.title("Temperaturas del suelo c/30min en Enero")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("Temp_30_Suelo_Enero",plt=2000)

In [ ]:
#Tomamos la temperatura del aire
graf3 = graf3.filter(['airT_Avg'],axis=1)
graf3=graf3.apply(pd.to_numeric,errors='coerce')
graf3.dtypes

In [ ]:
#Gráfica de temperatura del aire cada 30 min para enero del 2009
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); graf3.plot() ; plt.legend(loc='best')
plt.title("Temperatura del aire c/30 en Enero")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T30AireEnero",plt=2000)

In [ ]:
#Haremos de nuevo las gráficas para la temperatura del suelo a 4 profundidades y del aire aplicando el promedio móvil (rolling mean)

#Agregamos la variable fecha que desechamos anteriormente
df_10['FECHA']=df3['FECHA']
df_20['FECHA']=df3['FECHA']
df_40['FECHA']=df3['FECHA']
df_85['FECHA']=df3['FECHA']
graf2['FECHA']=df3['FECHA']


#Creamos nuevos data frames para los promedios móviles
df10roll = pd.DataFrame()
df20roll = pd.DataFrame()
df40roll = pd.DataFrame()
df85roll = pd.DataFrame()
dfairTroll = pd.DataFrame()


#Sacaremos el promedio móvil de cada una de las  4 temperaturas para las gráficas
df10roll["max10"]=df_10.T_10cm_max.rolling(60, min_periods=1).mean()
df10roll["min10"]=df_10.T_10cm_min.rolling(60, min_periods=1).mean()
df10roll["mean10"]=df_10.T_10cm_mean.rolling(60, min_periods=1).mean()

df20roll["max20"]=df_20.T_20cm_max.rolling(60, min_periods=1).mean()
df20roll["min20"]=df_20.T_20cm_min.rolling(60, min_periods=1).mean()
df20roll["mean20"]=df_20.T_20cm_mean.rolling(60, min_periods=1).mean()

df40roll["max40"]=df_40.T_40cm_max.rolling(60, min_periods=1).mean()
df40roll["min40"]=df_40.T_40cm_min.rolling(60, min_periods=1).mean()
df40roll["mean40"]=df_40.T_40cm_mean.rolling(60, min_periods=1).mean()

df85roll["max85"]=df_85.T_85cm_max.rolling(60, min_periods=1).mean()
df85roll["min85"]=df_85.T_85cm_min.rolling(60, min_periods=1).mean()
df85roll["mean85"]=df_85.T_85cm_mean.rolling(60, min_periods=1).mean()

dfairTroll["max_air"]=graf2.airT_Avg_max.rolling(60, min_periods=1).mean()
dfairTroll["min_air"]=graf2.airT_Avg_min.rolling(60, min_periods=1).mean()
dfairTroll["mean_air"]=graf2.airT_Avg_mean.rolling(60, min_periods=1).mean()


In [ ]:
#Gráfica suavizada 10cm 
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); df10roll.plot() ; plt.legend(loc='best')
plt.title("Temperatura del suelo - 10cm")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T_10roll",plt=2000)

In [ ]:
#Gráfica suavizada 20cm
fig=plt.figure(); df20roll.plot() ; plt.legend(loc='best')
plt.title("Temperatura del suelo - 20cm")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T_20roll",plt=2000)

In [ ]:
#Gráfica suavizada 40cm
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); df40roll.plot() ; plt.legend(loc='best')
plt.title("Temperatura del suelo - 40cm")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T_40roll",plt=2000)

In [ ]:
#Gráfica suavizada 85cm
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); df85roll.plot() ; plt.legend(loc='best')
plt.title("Temperatura del suelo - 85cm")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("T_85roll",plt=2000)

In [ ]:
#Gráfica suavizada de la T del aire
sns.set(rc={'figure.figsize':(12,8.27)})
fig=plt.figure(); dfairTroll.plot() ; plt.legend(loc='best')
plt.title("Temperatura del aire")
plt.ylabel("Temp ºC")
plt.grid(True)

fig.savefig("airT_roll",plt=2000)